In [53]:
import pandas as pd



In [54]:

df = pd.read_csv('/workspaces/vikrant_dubai/final/comb/UAX 1.csv')
unique_station_names = df['Station Name'].unique()


/tmp/ipykernel_25485/1482380921.py:1: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/workspaces/vikrant_dubai/final/comb/UAX 1.csv')


In [55]:
unique_station_names

array(['AL Maqtaa', 'Al Hamra'], dtype=object)

In [56]:
def create_finder_table(df):
    df['vlookup'] = df['Station Name'].str[:3] + df['Date'].str.replace('/', '')
    df_grouped = df.groupby(['Station Name', 'Date']).first().reset_index()
    df_grouped['vlookup'] = df_grouped['Station Name'].str[:5] + df_grouped['Date'].str.replace('/', '')
    Finder_table = df_grouped[['Station Name', 'Date', 'vlookup']]
    return Finder_table

# Example usage:
Finder_table = create_finder_table(df)
Finder_table.to_csv('/workspaces/vikrant_dubai/SQL/Finder_table.csv', index=False)

In [57]:


pollutants = ['SO2', 'NO2','CO','PM10', 'PM2.5']

# Convert pollutant columns to numeric, forcing errors to NaN
df[pollutants] = df[pollutants].apply(pd.to_numeric, errors='coerce')

avg_values = df.groupby(['Station Name', 'Date'])[pollutants].mean().reset_index()

final_df = pd.merge(avg_values, Finder_table, on=['Station Name', 'Date'], suffixes=(' Avg 24H', ''))
final_df.rename(columns={'Station Name': 'Name'}, inplace=True)
final_df['vlookup'] = final_df['Name'].str[:5] + final_df['Date'].str.replace('/', '')
final_df=final_df.drop(columns=['Name', 'Date'])
final_df



,SO2,NO2,CO,PM10,PM2.5,vlookup
0,6.587917,47.719167,NaN,24.620833,6.820606,AL Ma01012023
1,6.775600,26.989600,NaN,45.600000,11.896290,AL Ma01022023
2,7.490800,50.210800,NaN,42.428000,11.447957,AL Ma01032023
3,5.737917,17.447500,NaN,34.837500,9.637352,AL Ma01042023
4,7.488750,51.027083,NaN,188.933333,52.262270,AL Ma01052023
...,...,...,...,...,...,...
725,6.670000,34.716667,NaN,176.920833,47.721992,Al Ha31052023
726,5.938261,30.911739,NaN,82.952174,22.407407,Al Ha31072023
727,8.004167,42.055000,NaN,57.095833,15.552287,Al Ha31082023
728,9.463478,49.517826,NaN,61.873913,17.113043,Al Ha31102023


In [58]:



selected_columns = df[['Station Name', 'Date', 'O3']]

# Convert 'O3' column to numeric type
selected_columns['O3'] = pd.to_numeric(selected_columns['O3'], errors='coerce')


# Extract unique station names
unique_station_names = selected_columns["Station Name"].unique()

# Define the process_data function
def process_data(values, dates):
    # Create DataFrame
    df_temp = pd.DataFrame({'Index': range(len(values)), 'Values': values, 'Date': dates})

    # Calculate 8-hour moving average
    df_temp['8h_Avg'] = df_temp['Values'].rolling(window=8).mean()

    # Calculate datacap (percentage of available data in 8-hour window)
    df_temp['datacap'] = df_temp['Values'].rolling(window=8).count() / 8 * 100

    # Ensure first 7 values of 'datacap' are blank (NaN)
    #df_temp.loc[:6, 'datacap'] = None

    # Define function to calculate 8h_Avg_WHO and 8h_Avg_NS
    def calculate_adjusted_avg(df_temp, max_threshold):
        adjusted_avg = [None] * len(df_temp)
        for i in range(7, len(df_temp)):
            if df_temp.loc[i, 'datacap'] >= 75:
                adjusted_avg[i] = df_temp.loc[i, '8h_Avg']
            else:
                max_value = df_temp.loc[i-7:i, 'Values'].max()
                if max_value > max_threshold:
                    adjusted_avg[i] = max_value
        return adjusted_avg

    # Calculate 8h_Avg_WHO with max value threshold of 100
    df_temp['8h_Avg_WHO'] = calculate_adjusted_avg(df_temp, 100)

    # Calculate 8h_Avg_NS with max value threshold of 120
    df_temp['8h_Avg_NS'] = calculate_adjusted_avg(df_temp, 120)

    return df_temp

# Process data for each station name
dfs = []
for station in unique_station_names:
    station_data = selected_columns[selected_columns['Station Name'] == station]
    station_values = station_data["O3"].tolist()
    station_dates = station_data["Date"].tolist()
    processed_df = process_data(station_values, station_dates)
    processed_df['Station Name'] = station
    dfs.append(processed_df)

# Concatenate all DataFrames
df2 = pd.concat(dfs, ignore_index=True)

# Rearrange the columns as specified
df2 = df2[['Station Name', 'Date', 'Values', 'datacap', '8h_Avg_WHO', '8h_Avg_NS']]
df2.rename(columns={'Values': 'O3'}, inplace=True)

# Convert 'Date' column to datetime format for proper grouping
df2['Date'] = pd.to_datetime(df2['Date'], dayfirst=True)

# Calculate 8h_Avg_WHO_DC and 8h_Avg_NS_DC
df2['8h_Avg_WHO_DC'] = df2.groupby(['Station Name', 'Date'])['8h_Avg_WHO'].transform(lambda x: x.count() / 24 * 100)
df2['8h_Avg_NS_DC'] = df2.groupby(['Station Name', 'Date'])['8h_Avg_NS'].transform(lambda x: x.count() / 24 * 100)


/tmp/ipykernel_25485/3054381423.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['O3'] = pd.to_numeric(selected_columns['O3'], errors='coerce')


/tmp/ipykernel_25485/3054381423.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df2 = pd.concat(dfs, ignore_index=True)


In [59]:
df2

,Station Name,Date,O3,datacap,8h_Avg_WHO,8h_Avg_NS,8h_Avg_WHO_DC,8h_Avg_NS_DC
0,AL Maqtaa,2023-01-01,NaN,NaN,NaN,NaN,0.0,0.0
1,AL Maqtaa,2023-01-01,NaN,NaN,NaN,NaN,0.0,0.0
2,AL Maqtaa,2023-01-01,NaN,NaN,NaN,NaN,0.0,0.0
3,AL Maqtaa,2023-01-01,NaN,NaN,NaN,NaN,0.0,0.0
4,AL Maqtaa,2023-01-01,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
17517,Al Hamra,2023-12-31,11.25,100.0,39.93750,39.93750,100.0,100.0
17518,Al Hamra,2023-12-31,8.25,100.0,32.43750,32.43750,100.0,100.0
17519,Al Hamra,2023-12-31,7.25,100.0,25.12500,25.12500,100.0,100.0
17520,Al Hamra,2023-12-31,5.80,100.0,19.03750,19.03750,100.0,100.0


In [60]:

# Ensure only the last row of each day for each station retains the values, setting others to NaN
df2.loc[df2.duplicated(subset=['Station Name', 'Date'], keep='last'), ['8h_Avg_WHO_DC', '8h_Avg_NS_DC']] = None

# Group by 'Station Name' and 'Date'
grouped = df2.groupby(['Station Name', 'Date'])

# Create new columns for the calculated daily averages
df2['Daily_Avg_8h_WHO'] = None
df2['Daily_Avg_8h_NS'] = None

# Iterate over each group to calculate the averages where count > 75
for (station, date), group in grouped:
    if group['8h_Avg_WHO_DC'].sum() > 75:  # Check if total count > 75 for WHO
        avg_value_who = group['8h_Avg_WHO'].mean()  # Calculate average for WHO
        df2.loc[group.index[-1], 'Daily_Avg_8h_WHO'] = avg_value_who  # Assign to last row
    
    if group['8h_Avg_NS_DC'].sum() > 75:  # Check if total count > 75 for NS
        avg_value_ns = group['8h_Avg_NS'].mean()  # Calculate average for NS
        df2.loc[group.index[-1], 'Daily_Avg_8h_NS'] = avg_value_ns  # Assign to last row



# Convert 'Date' column to datetime format
df2['Date'] = pd.to_datetime(df2['Date'], dayfirst=True)

# Change date format to ddmmyyyy
df2['Date'] = df2['Date'].dt.strftime('%d/%m/%Y')

# Add a new column 'vlookup' that takes the first 5 letters of 'Station Name' and concatenates it with 'Date'
df2['vlookup'] = df2['Station Name'].str[:5] + df2['Date'].str.replace('/', '')

# Remove the 'Station Name' column
df2.drop(columns=['Station Name','Date'], inplace=True)


last_rows_per_date_station = df2.drop_duplicates(subset=['vlookup'], keep='last')

last_rows_per_date_station = last_rows_per_date_station[['vlookup'] + [col for col in last_rows_per_date_station.columns if col != 'vlookup']]

# Reset the index
last_rows_per_date_station.reset_index(drop=True, inplace=True)

last_rows_per_date_station



,vlookup,O3,datacap,8h_Avg_WHO,8h_Avg_NS,8h_Avg_WHO_DC,8h_Avg_NS_DC,Daily_Avg_8h_WHO,Daily_Avg_8h_NS
0,AL Ma01012023,NaN,0.0,NaN,NaN,0.0,0.0,None,None
1,AL Ma02012023,NaN,0.0,NaN,NaN,0.0,0.0,None,None
2,AL Ma03012023,NaN,0.0,NaN,NaN,0.0,0.0,None,None
3,AL Ma04012023,NaN,0.0,NaN,NaN,0.0,0.0,None,None
4,AL Ma05012023,NaN,0.0,NaN,NaN,0.0,0.0,None,None
...,...,...,...,...,...,...,...,...,...
726,Al Ha28122023,8.44,100.0,39.69000,39.69000,100.0,100.0,29.627604,29.627604
727,Al Ha29122023,12.14,100.0,41.80000,41.80000,100.0,100.0,33.95625,33.95625
728,Al Ha30122023,7.56,100.0,41.85125,41.85125,100.0,100.0,35.387292,35.387292
729,Al Ha31122023,5.80,100.0,19.03750,19.03750,100.0,100.0,27.371719,27.371719


In [61]:



selected_columns = df[['Station Name', 'Date', 'CO']]

df=selected_columns

# Convert 'Date' column to datetime format for proper processing
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Extract unique station names
unique_station_names = df["Station Name"].unique()

# Define the process_data function for CO transformation
def process_data(values, dates):
    # Create DataFrame
    df_temp = pd.DataFrame({'Index': range(len(values)), 'CO': values, 'Date': dates})

    # Calculate 8-hour moving average
    df_temp['C0_8h_Avg'] = df_temp['CO'].rolling(window=8).mean()

    # Calculate datacap (percentage of available data in 8-hour window)
    df_temp['datacap'] = df_temp['CO'].rolling(window=8).count() / 8 * 100

    # Ensure first 7 values of 'datacap' are blank (NaN)
    df_temp.loc[:6, 'datacap'] = None

    return df_temp

# Process data for each station name
dfs = []
for station in unique_station_names:
    station_data = df[df['Station Name'] == station]
    station_values = station_data["CO"].tolist()
    station_dates = station_data["Date"].tolist()
    processed_df = process_data(station_values, station_dates)
    processed_df['Station Name'] = station
    dfs.append(processed_df)

# Concatenate all DataFrames
df2 = pd.concat(dfs, ignore_index=True)

# Rearrange the columns as specified
df2 = df2[['Station Name', 'Date', 'CO', 'datacap', 'C0_8h_Avg']]

# Calculate C0_8h_Avg_DC (daily data capture percentage for CO 8h Avg)
df2['C0_8h_Avg_DC'] = df2.groupby(['Station Name', 'Date'])['C0_8h_Avg'].transform(lambda x: x.count() / 24 * 100)

# Ensure only the last row of each day for each station retains the values, setting others to NaN
df2.loc[df2.duplicated(subset=['Station Name', 'Date'], keep='last'), ['C0_8h_Avg_DC']] = None

# Group by 'Station Name' and 'Date'
grouped = df2.groupby(['Station Name', 'Date'])

# Create new columns for the calculated daily averages
df2['Daily_Avg_8h_C0'] = None

# Iterate over each group to calculate the daily averages where count > 75
for (station, date), group in grouped:
    if group['C0_8h_Avg_DC'].sum() > 75:  # Check if total count > 75
        avg_value_c0 = group['C0_8h_Avg'].mean()  # Calculate average
        df2.loc[group.index[-1], 'Daily_Avg_8h_C0'] = avg_value_c0  # Assign to last row


# Convert 'Date' column to datetime format
df2['Date'] = pd.to_datetime(df2['Date'], dayfirst=True)

# Change date format to ddmmyyyy
df2['Date'] = df2['Date'].dt.strftime('%d/%m/%Y')

# Add a new column 'vlookup' that takes the first 5 letters of 'Station Name' and concatenates it with 'Date'
df2['vlookup'] = df2['Station Name'].str[:5] + df2['Date'].str.replace('/', '')

# Remove the 'Station Name' column
df2.drop(columns=['Station Name','Date'], inplace=True)


last_rows_per_date_station2 = df2.drop_duplicates(subset=['vlookup'], keep='last')

last_rows_per_date_station2 = last_rows_per_date_station2[['vlookup'] + [col for col in last_rows_per_date_station2.columns if col != 'vlookup']]

# Reset the index
last_rows_per_date_station2.reset_index(drop=True, inplace=True)

last_rows_per_date_station2



,vlookup,CO,datacap,C0_8h_Avg,C0_8h_Avg_DC,Daily_Avg_8h_C0
0,AL Ma01012023,NaN,0.0,NaN,0.0,None
1,AL Ma02012023,NaN,0.0,NaN,0.0,None
2,AL Ma03012023,NaN,0.0,NaN,0.0,None
3,AL Ma04012023,NaN,0.0,NaN,0.0,None
4,AL Ma05012023,NaN,0.0,NaN,0.0,None
...,...,...,...,...,...,...
726,Al Ha28122023,NaN,0.0,NaN,0.0,None
727,Al Ha29122023,NaN,0.0,NaN,0.0,None
728,Al Ha30122023,NaN,0.0,NaN,0.0,None
729,Al Ha31122023,NaN,0.0,NaN,0.0,None


In [62]:



# Merge last_rows_per_date_station and last_rows_per_date_station2 on 'vlookup'
combined_df = pd.merge(last_rows_per_date_station, last_rows_per_date_station2, on='vlookup', suffixes=('_O3', '_CO'))

# Merge the result with final_df on 'vlookup'
combined_df = pd.merge(combined_df, final_df, on='vlookup')

# Display the combined dataframe
D24hparameter=combined_df[['vlookup','SO2', 'NO2', 'Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0', 'PM10', 'PM2.5']]

D24hparameter.to_csv('/workspaces/vikrant_dubai/SQL/D24hparameter.csv', index=False)




# Round all columns of D24hparameter to 1 decimal place
D24hparameter_rounded = D24hparameter.round(1)
# Convert 'Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', and 'Daily_Avg_8h_C0' to numeric, forcing errors to NaN
D24hparameter_rounded[['Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0']] = D24hparameter_rounded[['Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0']].apply(pd.to_numeric, errors='coerce')

# Round the specified columns to 1 decimal place
D24hparameter_rounded[['Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0']] = D24hparameter_rounded[['Daily_Avg_8h_WHO', 'Daily_Avg_8h_NS', 'Daily_Avg_8h_C0']].round(1)
# Display the new table
D24hparameter_rounded.to_csv('/workspaces/vikrant_dubai/SQL/D24hparameter_rounded.csv', index=False)    





df=D24hparameter_rounded

# Create a new DataFrame KPI_NS containing only the 'vlookup' column
KPI_NS = df[['vlookup']].copy()

# Add 'SO2' column based on the condition
KPI_NS['SO2'] = df['SO2'].apply(lambda x: 1 if x < 150.4 else 0)

# Add 'NO2' column based on the condition
KPI_NS['NO2'] = df['NO2'].apply(lambda x: 1 if x < 150.4 else 0)

# Add 'O3' column based on the condition using 'Daily_Avg_8h_NS'
KPI_NS['O3'] = df['Daily_Avg_8h_NS'].apply(lambda x: 1 if x < 120.4 else 0)

# Add 'CO' column based on the condition using 'Daily_Avg_8h_C0'
KPI_NS['CO'] = df['Daily_Avg_8h_C0'].apply(lambda x: 1 if x < 10.4 else 0)


KPI_NS.to_csv('/workspaces/vikrant_dubai/SQL/KPI_NS.csv', index=False)



df=D24hparameter_rounded

# Create a new DataFrame KPI_NS containing only the 'vlookup' column
KPI_WHO = df[['vlookup']].copy()

# Add 'SO2' column based on the condition
KPI_WHO['SO2'] = df['SO2'].apply(lambda x: 1 if x < 20.4 else 0)

# Add 'NO2' column based on the condition
KPI_WHO['NO2'] = df['NO2'].apply(lambda x: 1 if x < 75.4 else 0)

# Add 'O3' column based on the condition using 'Daily_Avg_8h_NS'
KPI_WHO['O3'] = df['Daily_Avg_8h_NS'].apply(lambda x: 1 if x < 100.4 else 0)

# Add 'CO' column based on the condition using 'Daily_Avg_8h_C0'
KPI_WHO['CO'] = df['Daily_Avg_8h_C0'].apply(lambda x: 1 if x < 10.4 else 0)


KPI_WHO.to_csv('/workspaces/vikrant_dubai/SQL/KPI_WHO.csv', index=False)




KPI_PM = df[['vlookup']].copy()
KPI_PM['PM10'] = df['PM10'].apply(lambda x: 1 if x < 150.4 else 0)
KPI_PM['PM2.5'] = df['PM2.5'].apply(lambda x: 1 if x < 60.4 else 0)
KPI_PM.to_csv('/workspaces/vikrant_dubai/SQL/KPI_PM.csv', index=False)